## IBM CAPSTOPNE PROJECT

### INSTALL BeautifulSoap

In [67]:
!pip install watson-developer-cloud==1.5

In [68]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Fetch Data

In [69]:
urlx="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(urlx).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#get the table
ziptable = soup.find("table", attrs={"class": "wikitable"})
# get the rows in the body
zipdata = ziptable.tbody.find_all("tr") 
# Get all the headings of Lists
headings = []
for td in zipdata[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

#prepare data frame    
df = pd.DataFrame(columns = headings, index= range(0,len(zipdata)-1))

#collect table data into datafram
row_marker = 0
for row in zipdata:
    column_marker = 0
    columns = row.find_all("td")
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if len(columns) >0:
        row_marker += 1
#check data frame
df.head(10)               

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


### Edit Data

In [70]:
#apply Borough requirement
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]
df2.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


In [71]:
#apply Neighbourhood requirement                 
df2['Neighbourhood'] = np.where(df2['Neighbourhood'].str.contains('Not assigned', na=False),  df2['Borough'], df2['Neighbourhood'])
df2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


### CHECK The Shape

In [65]:
df2.shape

(211, 3)